In [286]:
#importing libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas
import math
from pandas import DataFrame
from matplotlib import pyplot


In [113]:
#importing dataset
iotdata = pd.read_csv('data/plot2-IoT Sensor Data.csv')
tf_final = iotdata.sort_values('datetime')        #aligning the timestamps of all data
tf_final.set_index('datetime')                    #setting the index of date and time

,sensor,value
datetime,,
2019-01-01 00:26:26.749000+00:00,HUM,100.00
2019-01-01 00:26:26.749000+00:00,TC,5.92
2019-01-01 00:26:26.749000+00:00,PRES,98650.55
2019-01-01 01:26:26.650000+00:00,HUM,100.00
2019-01-01 01:26:26.650000+00:00,TC,5.55
...,...,...
2019-12-31 22:39:37.032000+00:00,HUM,100.00
2019-12-31 22:39:37.032000+00:00,PRES,98181.39
2019-12-31 23:39:37.185000+00:00,PRES,98217.69


In [114]:
#selecting date time column
datetime = tf_final['datetime']
datetime_f = datetime.reset_index(drop=True)

In [115]:
#removing the format which is after seconds. 
from datetime import datetime

def time_conv(dataset):
    row_len = len(datetime_f)
    datetime = []
    for i in range(row_len):
        strr = datetime_f[i].split('.')[0]
        datetime.append(strr)
    return datetime

x = time_conv(datetime_f)
tf_final['date_time'] = x
dataset = tf_final.drop(['datetime'], axis = 1)
dataset.set_index('date_time')

,sensor,value
date_time,,
2019-01-01 00:26:26,HUM,100.00
2019-01-01 00:26:26,TC,5.92
2019-01-01 00:26:26,PRES,98650.55
2019-01-01 01:26:26,HUM,100.00
2019-01-01 01:26:26,TC,5.55
...,...,...
2019-12-31 22:39:37,HUM,100.00
2019-12-31 22:39:37,PRES,98181.39
2019-12-31 23:39:37,PRES,98217.69


In [117]:
#converting dataset into more customed dataframe for ease.
data = tf_final.pivot_table('value', ['date_time'], 'sensor')

#converting Pressure unit from pascal to kilo pascal.
data['PRES'] /= 1000


data

sensor,HUM,PRES,TC
date_time,,,
2019-01-01 00:26:26,100.00,98.65055,5.92
2019-01-01 01:26:26,100.00,98.73618,5.55
2019-01-01 02:26:27,100.00,98.82278,12.93
2019-01-01 03:26:26,89.86,98.87944,19.03
2019-01-01 04:26:27,36.46,98.86911,24.00
...,...,...,...
2019-12-31 19:39:36,100.00,98.21567,17.09
2019-12-31 20:39:36,100.00,98.23615,16.87
2019-12-31 21:39:36,100.00,98.18532,16.23


In [249]:
#Scaling the values of dataframe in the range of 0 and 1
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(0, 1))
training_set_scaled = scaler.fit_transform(data)

columns = ['HUM', 'PRES', 'TC']
dataset = DataFrame(training_set_scaled, columns=columns)

print(dataset)

         HUM      PRES        TC
0     1.0000  0.595639  0.070323
1     1.0000  0.612752  0.061650
2     1.0000  0.630058  0.234646
3     0.8986  0.641382  0.377637
4     0.3646  0.639317  0.494140
...      ...       ...       ...
8317  1.0000  0.508730  0.332161
8318  1.0000  0.512823  0.327004
8319  1.0000  0.502665  0.312002
8320  1.0000  0.501880  0.299109
8321  1.0000  0.509134  0.300047

[8322 rows x 3 columns]


In [247]:
#X for input to LSTM model as a training data and y_temp, y_pres, y_hum given one by one to model as an input. 
X = dataset.iloc[:,:].to_numpy()

y_temp = dataset['TC'].to_numpy()
y_pres = dataset['PRES'].to_numpy()
y_hum = dataset['HUM'].to_numpy()

In [293]:
#For Temperature Prediction
x_arr = []
y_arr = []
for i in range(X.shape[0]-(120+168)):
    x_arr.append(X[i:i+168])
    y_arr.append(y_temp[i+168:i+168+120])
    
x_arr = np.array(x_arr)
y_arr = np.array(y_arr)

train_ind = int(0.8 * x_arr.shape[0])

x_train = x_arr[:train_ind]
x_test = x_arr[train_ind:]
y_temp_train = y_arr[:train_ind]
y_temp_test = y_arr[train_ind:]

print(x_temp_train.shape, x_temp_test.shape,y_temp_train.shape, y_temp_test.shape)


(6427, 168, 3) (1607, 168, 3) (6427, 120) (1607, 120)


In [294]:
#For Pressure Prediction
x_arr = []
y_arr = []
for i in range(X.shape[0]-(120+168)):
    x_arr.append(X[i:i+168])
    y_arr.append(y_pres[i+168:i+168+120])
    
x_arr = np.array(x_arr)
y_arr = np.array(y_arr)

train_ind = int(0.8 * x_arr.shape[0])

x_train = x_arr[:train_ind]
x_test = x_arr[train_ind:]
y_pres_train = y_arr[:train_ind]
y_pres_test = y_arr[train_ind:]

print(x_train.shape, x_test.shape,y_pres_train.shape, y_pres_test.shape)

(6427, 168, 3) (1607, 168, 3) (6427, 120) (1607, 120)


In [295]:
#For Humidity Prediction
x_arr = []
y_arr = []
for i in range(X.shape[0]-(120+168)):
    x_arr.append(X[i:i+168])
    y_arr.append(y_hum[i+168:i+168+120])
    
x_arr = np.array(x_arr)
y_arr = np.array(y_arr)

train_ind = int(0.8 * x_arr.shape[0])

x_train = x_arr[:train_ind]
x_test = x_arr[train_ind:]
y_hum_train = y_arr[:train_ind]
y_hum_test = y_arr[train_ind:]

print(x_train.shape, x_test.shape,y_hum_train.shape, y_hum_test.shape)

(6427, 168, 3) (1607, 168, 3) (6427, 120) (1607, 120)


In [287]:
#LSTM VANILLA MODEL
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras import optimizers 
import sklearn.metrics as metrics

model = Sequential()
model.add(LSTM(50, input_shape=(168, 3), return_sequences=False))
model.add(Dense(units=120))

model.compile(optimizer='adam', loss='mse')

In [310]:
## For Temperature
# Model Fitting for Temperature
model.fit(x_train, y_temp_train, epochs=10)

# Model Prediction for temperature
test_temp_predict = model.predict(x_test)

# To find the error between predicted and actual values. RMSE is used as a cost function. 
scaler_temps = MinMaxScaler().fit(data['TC'].to_numpy().reshape(-1,1))
tp1 = scaler_temps.inverse_transform(test_temp_predict.reshape(-1,1))
yt1 = scaler_temps.inverse_transform(y_temp_test.reshape(-1,1))

Epoch 1/10
201/201 [==============================] - 16s 78ms/step - loss: 0.0149
Epoch 2/10
201/201 [==============================] - 14s 69ms/step - loss: 0.0065
Epoch 3/10
201/201 [==============================] - 16s 81ms/step - loss: 0.0062
Epoch 4/10
201/201 [==============================] - 16s 78ms/step - loss: 0.0060
Epoch 5/10
201/201 [==============================] - 14s 69ms/step - loss: 0.0059
Epoch 6/10
201/201 [==============================] - 15s 77ms/step - loss: 0.0058
Epoch 7/10
201/201 [==============================] - 15s 72ms/step - loss: 0.0058
Epoch 8/10
201/201 [==============================] - 14s 71ms/step - loss: 0.0057
Epoch 9/10
201/201 [==============================] - 16s 79ms/step - loss: 0.0056
Epoch 10/10
201/201 [==============================] - 17s 83ms/step - loss: 0.0056


In [304]:
# Error Value in Percentage of Temperature
np.mean(np.abs((yt1-tp1)/yt1))*100


23.32263707736011

In [305]:
## For Humidity
# Model Fitting for Humidity
model.fit(x_train, y_hum_train, epochs=10)

# Model Prediction for Humidity
test_hum_predict = model.predict(x_test)

# To find the error between predicted and actual values. RMSE is used as a cost function. 
scaler_temps = MinMaxScaler().fit(data['HUM'].to_numpy().reshape(-1,1))
tp2 = scaler_temps.inverse_transform(test_hum_predict.reshape(-1,1))
yt2 = scaler_temps.inverse_transform(y_hum_test.reshape(-1,1))

Epoch 1/10
201/201 [==============================] - 17s 86ms/step - loss: 0.0578
Epoch 2/10
201/201 [==============================] - 15s 74ms/step - loss: 0.0321
Epoch 3/10
201/201 [==============================] - 14s 71ms/step - loss: 0.0306
Epoch 4/10
201/201 [==============================] - 14s 71ms/step - loss: 0.0298
Epoch 5/10
201/201 [==============================] - 15s 73ms/step - loss: 0.0290
Epoch 6/10
201/201 [==============================] - 14s 71ms/step - loss: 0.0285
Epoch 7/10
201/201 [==============================] - 14s 72ms/step - loss: 0.0277
Epoch 8/10
201/201 [==============================] - 14s 70ms/step - loss: 0.0270
Epoch 9/10
201/201 [==============================] - 16s 81ms/step - loss: 0.0265
Epoch 10/10
201/201 [==============================] - 17s 84ms/step - loss: 0.0251


In [306]:
# Error Value in Percentage of Humidity
np.mean(np.abs((yt2-tp2)/yt2))*100


28.741437452110922

In [307]:
## For Pressure
# Model Fitting for Pressure
model.fit(x_train, y_pres_train, epochs=10)

# Model Prediction for Pressure
test_pres_predict = model.predict(x_test)

# To find the error between predicted and actual values. RMSE is used as a cost function. 
scaler_temps = MinMaxScaler().fit(data['PRES'].to_numpy().reshape(-1,1))
tp3 = scaler_temps.inverse_transform(test_pres_predict.reshape(-1,1))
yt3 = scaler_temps.inverse_transform(y_pres_test.reshape(-1,1))

Epoch 1/10
201/201 [==============================] - 19s 95ms/step - loss: 0.0128
Epoch 2/10
201/201 [==============================] - 16s 81ms/step - loss: 0.0030
Epoch 3/10
201/201 [==============================] - 22s 111ms/step - loss: 0.0029
Epoch 4/10
201/201 [==============================] - 17s 83ms/step - loss: 0.0028
Epoch 5/10
201/201 [==============================] - 15s 73ms/step - loss: 0.0028
Epoch 6/10
201/201 [==============================] - 16s 79ms/step - loss: 0.0027
Epoch 7/10
201/201 [==============================] - 16s 82ms/step - loss: 0.0027
Epoch 8/10
201/201 [==============================] - 21s 104ms/step - loss: 0.0027
Epoch 9/10
201/201 [==============================] - 15s 74ms/step - loss: 0.0026
Epoch 10/10
201/201 [==============================] - 18s 89ms/step - loss: 0.0026


In [337]:
test_pres_predict.shape

(1607, 120)

In [308]:
# Error Value in Percentage of Humidity
np.mean(np.abs((yt3-tp3)/yt3))*100

0.21392863140958235

In [333]:
p_temp = tp1.flatten()
p_hum = tp2.flatten()
p_pres = tp3.flatten()

print(p_temp.shape, p_hum.shape, p_pres.shape)

(192840,) (192840,) (192840,)


In [345]:
# Creating the dataframe of predicted values

df = pd.DataFrame() 
df['pred_tc'] = p_temp
df['pred_hum'] = p_hum
df['pred_pres'] = p_pres

In [346]:
df

,pred_tc,pred_hum,pred_pres
0,26.057100,104.031944,97.855804
1,24.940907,104.210007,97.917290
2,24.575678,102.319572,97.904526
3,24.362282,101.616478,97.929611
4,24.171061,100.263893,97.920792
...,...,...,...
192835,14.179567,98.732719,98.312241
192836,14.393075,95.630150,98.311859
192837,13.854380,93.652687,98.359695
192838,14.516295,94.381561,98.306534
